In [1]:
import os
import sys
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
import sys
sys.path.append("../")

In [2]:
import sys
sys.path.append("../")
from typing import Any, Dict
import src.requestcompletion as rc
from src.requestcompletion.llm import MessageHistory, UserMessage
from src.requestcompletion.visuals.agent_viewer import AgentViewer
from src.requestcompletion.nodes.library import from_function

In [3]:
from demo.sample_tools.word_tools import check_chars

class CheckAgenticTool(rc.library.ToolCallLLM):
    def __init__(self, message_history: MessageHistory):
        super().__init__(message_history=message_history, 
                         model=rc.llm.OpenAILLM("gpt-4o"),  # or any other model you want to use
                         )

    # define a set of tools that the agent can use
    def connected_nodes(self):  
        return {
            from_function(check_chars)
            }
    
    # =================== If this node is a tool, then it needs to implement the following ==================
    @classmethod
    def tool_info(cls) -> rc.llm.Tool:
        return rc.llm.Tool(
            name="CheckAgenticTool",
            detail="Check chars",
            parameters={rc.llm.Parameter(name="check_chars_request", param_type="string", description="The detailed task prompt for the task to be completed.")},
        )

    @classmethod
    def prepare_tool(cls, tool_parameters: Dict[str, Any]):
        message_hist = MessageHistory(
            [UserMessage(f"Request Prompt: '{tool_parameters['check_chars_request']}'")]
        )
        return cls(message_hist)
    # ===========================================================================================================

    @classmethod
    def pretty_name(cls) -> str:
        return "Check Agentic Tool"



In [4]:
# Import (or write) the functions that we want to give to the agent as tools
from demo.sample_tools.word_tools import check_chars
word_check_node = rc.library.from_function(check_chars)
 

In [5]:
WordAgent = rc.library.tool_call_llm(
    connected_nodes={
        word_check_node
    },
    pretty_name="Word Agent",
    system_message=rc.llm.SystemMessage(
        "You are a helpful ai that counts the occurence of certain charactes in words given to you. You will have access to WordCheckNode and should always use this."
    ),
    model=rc.llm.OpenAILLM("gpt-4o"),  # you can use any model you want
)

In [6]:
USER_PROMPT = "The number of r's in Strawberry"

In [7]:

with rc.Runner(executor_config=rc.ExecutorConfig(timeout=50)) as runner:
    result = await runner.run(WordAgent, message_history=MessageHistory([UserMessage(USER_PROMPT)]))
# async def top_level_node(user_prompt: str):
    
#     result = await rc.call(WordAgent, message_history=MessageHistory([UserMessage(user_prompt)]))

# VoiceNotionAgent = from_function(top_level_node(USER_PROMPT))

[+21.961 s] RC.RUNNER   : INFO     - START CREATED Word Agent - (, message_history=user: The number of r's in Strawberry)
[+23.540 s] RC.RUNNER   : INFO     - Word Agent CREATED check_chars Node - ({'input': 'Strawberry', 'char': 'r'}, )
[+23.541 s] RC.RUNNER   : INFO     - check_chars Node DONE 3
[+24.476 s] RC.RUNNER   : INFO     - Word Agent DONE assistant: The word "Strawberry" contains 3 instances of the letter 'r'.
